# Imports

In [1]:
from keras import layers
from keras import models
import pandas as pd
import numpy as np
import pickle
from keras import regularizers

Using TensorFlow backend.


# Functions

In [2]:
def build_model_R(units,layerz,metric,shape,lozz= 'logcosh'):
    model = models.Sequential()
    model.add(layers.Dense(units, activation = 'relu', 
                           input_shape = (shape,)))
    for layer in range(layerz):
        model.add(layers.Dense(units, activation = 'relu'))
    model.add(layers.Dense(1)) #linear layer
    model.compile(optimizer = 'rmsprop', loss = lozz, metrics =[metric] )
    return model

def build_model_C(units,layerz,metric,shape,lozz = 'binary_crossentropy'):
    model = models.Sequential()
    model.add(layers.Dense(units, activation = 'relu', 
                           kernel_regularizer = regularizers.l2(0.001),
                           input_shape = (shape,)))
    for layer in range(layerz):
        model.add(layers.Dropout(0.4))
        model.add(layers.Dense(units, activation = 'relu'))
    model.add(layers.Dense(1, activation = "sigmoid")) 
    model.compile(optimizer = 'Adam', loss = lozz, metrics =[metric] )
    return model

def preprocess(path):    
    pair_df = pd.read_csv(path)
    pair_df.drop(columns = "dateorder", inplace = True)
    data = pair_df.loc[:,'inter1_x':'fif12'].values.astype("float")
    labels = pair_df.loc[:,"liked":].values.astype("float")
    
    # remove a column
    
    # randomize data and label set
    sample_size = data.shape[0]
    arr = np.arange(sample_size)
    data = data[arr].reshape(data.shape)
    labels = labels[arr].reshape(labels.shape)
    
    dependent_measures = dict(
    liked = labels[:,0], sexatt = labels[:,1], likyes = labels[:,2],
    fliked = labels[:,3], fsexatt = labels[:,4] , flikyes = labels[:,5],
    saidyes = labels[:,6], fsaidyes = labels[:,6], match = labels[:,7]
    )

    # normalize
    mean = data.mean(axis = 0)
    std = data.std(axis = 0)

    data -= mean
    data /= std
    
    return data, dependent_measures,mean,std


path = "./pair2Dataframe.csv"
data, dependent_measures,mean,std  = preprocess(path)

def save_model( measure, model , X = data ,units = 64, layerz = 5, num_epochs = 10):#

    y = dependent_measures[measure]
    shape = X.shape[1]
    #Hyperparameters                                

    # create model
    build = dict( regression = build_model_R(units,layerz,'mae',shape),
                  classification = build_model_C(units,layerz,"accuracy",shape)
    )
    model = build[model]
    # train model
    model.fit(X, y, epochs = num_epochs, batch_size = 500, verbose = 0 )
    # Saving model
    file = measure+'Model.h5'
    print(file)
    model.save(file) 

In [12]:
save_model("saidyes","classification")
save_model("fsaidyes","classification")
save_model("match","classification")

saidyesModel.h5
fsaidyesModel.h5
matchModel.h5


In [3]:
#save_model("liked","regression")
#save_model("sexatt","regression")
#save_model("likyes","regression")
save_model("fliked","regression")
#save_model("fsexatt","regression")
#save_model("flikyes","regression")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
flikedModel.h5


In [ ]:
hyperparameters = dict(mean= mean, std= std)

In [ ]:
pickle_out = open("meanAndstd","wb")
pickle.dump(hyperparameters,pickle_out)
pickle_out.close()